## SAGE - Self Adaptive Goal-Oriented Execution Agent  
SAGE is an advanced AI agent system based on the Self-Adaptive Goal-oriented Execution framework, using Google’s Gemini API. It defines 4 roles and uses its AI model to run these in a self-adapting cycle.

Let's start by loading the libraries we'll need

In [1]:
from google import genai
from google.genai import types
from google.colab import userdata
import json
import time
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, asdict
from enum import Enum

We now need to set up our wrap-around CSS function.

In [ ]:
from IPython.display import display, HTML
def set_css():
  display(HTML('''
    <style>
      pre {white-space: pre-wrap;}
    </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

Let's create a set of task status codes which we use to set task status as the tasks run.

In [2]:
class TaskStatus(Enum):
   PENDING = "pending"
   IN_PROGRESS = "in_progress"
   COMPLETED = "completed"
   FAILED = "failed"

Let's create the class of Task. We'll be generating tasks and running them to achieve our goal.

In [3]:
@dataclass
class Task:
   id: str
   description: str
   priority: int
   status: TaskStatus = TaskStatus.PENDING
   dependencies: List[str] = None
   result: Optional[str] = None

   def __post_init__(self):
       if self.dependencies is None:
           self.dependencies = []

We'll now create our Agent as a class, and set up a number of model calls within that class which together provide agentic capability. We'll be using four functions each making a model call with a role-specific context:  
- self_assess  
- adaptive_plan  
- execute_goal_oriented  
- integrate_experience  
  
We'll be using the same model for each in this demonstration. We can run multiple iterations and retain memory between iterations. We'll set up some class-wide variables for that. We'll include tracking of tokens used to indicate the relative cost of agentic AI.

In [4]:
class SAGEAgent:
   """Self-Adaptive Goal-oriented Execution AI Agent"""

   def __init__(self, api_key: str, model_name: str = "gemini-2.0-flash"):
       self.client = genai.Client(api_key=api_key)
       self.ground_tool = types.Tool(google_search=types.GoogleSearch())
       self.ground_config  = types.GenerateContentConfig(tools=[self.ground_tool])
       self.ground_model   = model_name
       self.memory = []
       self.tasks = {}
       self.context = {}
       self.augment = ""
       self.tokens = 0
       self.iteration_count = 0

# ---------------------------------------------------------------------
# We'll define the helper functions we need to support the SAGE approach.
# There's a function to clear the response of any ticks or json prefixes
#
   def tidy(self,text) -> str:
      if text.startswith('```'):
          text = text.split('```')[1]
          if text.startswith('json'):
              text = text[4:]
      text = text.strip()
      return text

# A function to check if we've met all dependencies
   def _dependencies_met(self, task: Task) -> bool:
       """Check if task dependencies are satisfied"""
       for dep_id in task.dependencies:
           if dep_id not in self.tasks or self.tasks[dep_id].status != TaskStatus.COMPLETED:
               return False
       return True

# A function to update the context based on task results.
   def _update_context(self, results: List[Dict[str, Any]]):
       """Update agent context based on execution results"""
       completed_tasks = [r for r in results if r["task"]["status"] == "completed"]
       self.context.update({
           "completed_tasks": len(completed_tasks),
           "total_tasks": len(self.tasks),
           "success_rate": len(completed_tasks) / len(results) if results else 0,
           "last_update": time.time()
       })

#--------------------------------------------------------------------------------------------------
# **Self Assess** is the first of the four model-based functions. It reviews the context and
# assesses our overall level of completeness based on task completion.  The first thing we do is to
# set up an assessment prompt and pass it with our current context into the GenAI model requesting
# a self-assessment. The function returns in JSON form a dictionary as defined.
   def self_assess(self, goal: str, context: Dict[str, Any]) -> Dict[str, Any]:
       """S: Self-Assessment - Evaluate current state and capabilities"""

       assessment_prompt = f"""
       You are an AI agent conducting self-assessment to determine the progress you are
       making in reaching the goal. Calculate the progress score by considering the
       level of detail on each task completed, whether more detail can be added, and
       whether more steps are recommended.
       Respond ONLY with valid JSON, no additional text.
       GOAL: {goal}
       CONTEXT: {json.dumps(context, indent=2)}
       TASKS_PROCESSED: {len(self.tasks)}
       Provide assessment as JSON with these exact keys:
       {{
           "progress_score": <number 0-100>,
           "resources": ["list of available resources"],
           "gaps": ["list of knowledge gaps"],
           "risks": ["list of potential risks"],
           "recommendations": ["list of next steps"]
       }}
       """

       response = self.client.models.generate_content(
           model=self.ground_model,
           contents=assessment_prompt,
           config=self.ground_config)
       self.tokens = self.tokens + response.usage_metadata.total_token_count
       try:
           text = self.tidy(response.text.strip())
           return json.loads(text)
       except Exception as e:
           print(f"Assessment parsing error: {e}")
           return {
               "progress_score": 25,
               "resources": ["AI capabilities", "Internet knowledge"],
               "gaps": ["Specific domain expertise", "Real-time data"],
               "risks": ["Information accuracy", "Scope complexity"],
               "recommendations": ["Break down into smaller tasks", "Focus on research first"]
           }

#------------------------------------------------------------------------------------------
# **Adaptive_Plan** The second function is the adaptive planning which breaks down the goal into
# 3-4 tasks per iteration. We call the GenAI model and get the response in JSON form, and then
# extract the tasks into the tasks array.
   def adaptive_plan(self, goal: str, assessment: Dict[str, Any]) -> List[Task]:
       """A: Adaptive Planning - Create dynamic, context-aware task decomposition"""

       planning_prompt = f"""
       You are an AI task planner. Respond ONLY with valid JSON array, no additional text.
       MAIN_GOAL: {goal}
       ASSESSMENT: {json.dumps(assessment, indent=2)}
       Create 4-5 actionable tasks as JSON array:
       [
           {{
               "id": "task_1",
               "description": "Clear, specific task description",
               "priority": 5,
               "dependencies": []
           }},
           {{
               "id": "task_2",
               "description": "Another specific task",
               "priority": 4,
               "dependencies": ["task_1"]
           }}
       ]
       Each task must have: id (string), description (string), priority (1-5), dependencies (array of strings)
       """

       response = self.client.models.generate_content(
           model=self.ground_model,
           contents=planning_prompt,
           config=self.ground_config)
       self.tokens = self.tokens + response.usage_metadata.total_token_count
       try:
           text = self.tidy(response.text.strip())
           task_data = json.loads(text)
           tasks = []
           for i, task_info in enumerate(task_data):
               task = Task(
                   id=task_info.get('id', f'task_{i+1}'),
                   description=task_info.get('description', 'Undefined task'),
                   priority=task_info.get('priority', 3),
                   dependencies=task_info.get('dependencies', [])
               )
               tasks.append(task)
           return tasks
       except Exception as e:
           print(f"Planning parsing error: {e}")
           return [
               Task(id="research_1", description="Research sustainable urban gardening basics", priority=5),
               Task(id="research_2", description="Identify space-efficient growing methods", priority=4),
               Task(id="compile_1", description="Organize findings into structured guide", priority=3, dependencies=["research_1", "research_2"])
           ]

#-------------------------------------------------------------------------------------
# **Goal Oriented Execution** The next function is used to executes a task based on the goal and
# the current context. The AI model is directed to break the task into steps and execute
# each step. The results are added to the augmentation information to be used to write
# the report.
   def execute_goal_oriented(self, task: Task) -> str:
       """G: Goal-oriented Execution - Execute specific task with focused attention"""

       execution_prompt = f"""
       GOAL-ORIENTED EXECUTION:
       Task: {task.description}
       Priority: {task.priority}
       Context: {json.dumps(self.context, indent=2)}
       Execute this task step-by-step:
       1. Break down the task into concrete actions
       2. Execute each action methodically
       3. Validate results at each step
       4. Provide comprehensive output
       Focus on practical, actionable results. Be specific and thorough.
       """

       response = self.client.models.generate_content(
           model=self.ground_model,
           contents=execution_prompt,
           config=self.ground_config)
       self.tokens = self.tokens + response.usage_metadata.total_token_count
       return response.text.strip()

#---------------------------------------------------------------------------------------
# **Integrate_Experience** The final function calls a model to integrate the task and
# the results of executing it with learnings from the current experience.
   def integrate_experience(self, task: Task, result: str, success: bool) -> Dict[str, Any]:
       """E: Experience Integration - Learn from outcomes and update knowledge"""

       integration_prompt = f"""
       You are learning from task execution. Respond ONLY with valid JSON, no additional text.
       TASK: {task.description}
       RESULT: {result[:200]}...
       SUCCESS: {success}
       Provide learning insights as JSON:
       {{
           "learnings": ["key insight 1", "key insight 2"],
           "patterns": ["pattern observed 1", "pattern observed 2"],
           "adjustments": ["adjustment for future 1", "adjustment for future 2"],
           "confidence_boost": <number -10 to 10>
       }}
       """

       response = self.client.models.generate_content(
           model=self.ground_model,
           contents=integration_prompt,
           config=self.ground_config)
       self.tokens = self.tokens + response.usage_metadata.total_token_count
       try:
           text = self.tidy(response.text.strip())
           experience = json.loads(text)
           experience['task_id'] = task.id
           experience['timestamp'] = time.time()
           self.memory.append(experience)
           return experience
       except Exception as e:
           print(f"Experience parsing error: {e}")
           experience = {
               "learnings": [f"Completed task: {task.description}"],
               "patterns": ["Task execution follows planned approach"],
               "adjustments": ["Continue systematic approach"],
               "confidence_boost": 5 if success else -2,
               "task_id": task.id,
               "timestamp": time.time()
           }
           self.memory.append(experience)
           return experience

#--------------------------------------------------------------------------------
# The sage_out function is used to generate the final report to satisfy the goal
# based on the aggregated augmentation information, and using glm4's own built-in
# web search capability.
   def sage_out(self, goal: str):
       """Create the final report"""
       print(f" Creating Final Report")

       report_prompt = f"""
       You are a Report Writer who can take RESEARCH information and write a well structured report to
       meet the GOAL provided. Respond using an essay format report with detailed information for each
       point covered in the research. Search the web to get additional details to flesh out the report
       and provide examples for all key bullet points.
       GOAL: {goal}
       RESEARCH: {self.augment}
       """
       response = self.client.models.generate_content(
           model=self.ground_model,
           contents=report_prompt,
           config=self.ground_config)
       self.tokens = self.tokens + response.usage_metadata.total_token_count
       return response.text.strip()

# The final routine controls the calls to the models, providing the agentic flow
# through the agent.
   def execute_sage_cycle(self, goal: str, max_iterations: int = 3) -> Dict[str, Any]:
       """Execute complete SAGE cycle for goal achievement"""
       print(f"SAGE analysis for goal: {goal}")
       results = {"goal": goal, "iterations": [], "final_status": "unknown"}

       assessment = self.self_assess(goal, self.context)
       for iteration in range(max_iterations):
           self.iteration_count += 1
           print(f"\nSAGE Iteration {iteration + 1}")

           tasks = self.adaptive_plan(goal, assessment)
           print(f" Adaptive Planning generated {len(tasks)} tasks")

           print(" Goal-oriented Execution...")
           iteration_results = []

           for task in sorted(tasks, key=lambda x: x.priority, reverse=True):
               if self._dependencies_met(task):
                   task.status = TaskStatus.IN_PROGRESS

                   try:
                       result = self.execute_goal_oriented(task)
                       task.result = result
                       self.augment = self.augment + result.strip()
                       task.status = TaskStatus.COMPLETED
                       success = True
                       print(f"   Completed: {task.description}")
                   except Exception as e:
                       task.status = TaskStatus.FAILED
                       task.result = f"Error: {str(e)}"
                       success = False
                       print(f"   Failed: {task.description}")

                   print(" Integrating learned experience...")
                   experience = self.integrate_experience(task, task.result, success)

                   self.tasks[task.id] = task
                   iteration_results.append({
                       "task": asdict(task),
                       "experience": experience
                   })

           self._update_context(iteration_results)

           results["iterations"].append({
               "iteration": iteration + 1,
               "assessment": assessment,
               "tasks_generated": len(tasks),
               "tasks_completed": len([r for r in iteration_results if r["task"]["status"] == "completed"]),
               "results": iteration_results
           })

           assessment = self.self_assess(goal, self.context)
           print(f" Self-Assessment Score: {assessment.get('progress_score', 0)}/100")
           if assessment.get('progress_score', 0) >= 90:
               results["final_status"] = "achieved"
               print(f"\nSAGE goal achieved at a cost of {self.tokens} tokens!")
               break

       if results["final_status"] == "unknown":
           results["final_status"] = "in_progress"

       print(f"\nSAGE Report Generation...")
       report = self.sage_out(goal)
       with open("Sage_Report","w") as f:
          f.write(report)

       return results

We can now set our goal and then call the SAGE controller routine from our mainline.


In [ ]:
if __name__ == "__main__":
   API_KEY = userdata.get("GEMINI_API_KEY")
   agent = SAGEAgent(API_KEY, model_name="gemini-2.0-flash")
   goal = "Research and create a comprehensive guide on astrophotography"
   print("\n" + "="*50)
   print(" SAGE EXECUTION SUMMARY")
   print("="*50)
   try:
       results = agent.execute_sage_cycle(goal, max_iterations=4)
   except Exception as e:
       print(f"Error encountered: {e}")

The results data structure contains three elements: goal, iterations, and final_status. The information researched is in the iterations item which is one entry for each iteration carried out. Each iteration has five elements: iteration, assessment, tasks_generated, tasks_completed, and results. The results item holds results for each task with elements task and experience. The task item has six elements: id, description, priority, status, dependencies, and the result which we focus on.